In [2]:
!pip install langchain-core langchain-community pymupdf

In [ ]:
from langchain_core.documents import Document

In [ ]:
doc=Document(
    page_content="Hello world",
    metadata={
        "source":"example.txt",
        "pages":1,
        "author":"Aashik Thakur",
        "date_created":"2025-12-17"
    }
)
doc

In [ ]:
### TextLoader
from langchain_community.document_loaders import TextLoader
loader=TextLoader("text-files/ai.txt",encoding="utf-8")
document=loader.load()
print(document)


In [ ]:
### directory loader
from langchain_community.document_loaders import DirectoryLoader

## load all the files from a directory
dir_loader=DirectoryLoader(
    "text-files",
    glob="**/*.txt", ##pattern to match files
    loader_cls=TextLoader, ## loader class to use
    loader_kwargs={"encoding":"utf-8"}
)
documents=dir_loader.load()
documents

In [ ]:
### load a pdf file
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

dir_loader=DirectoryLoader(
    "pdf-files",
    glob="**/*.pdf", ##pattern to match files
    loader_cls=PyMuPDFLoader ## loader class to use
)

pdf_documents = dir_loader.load()
pdf_documents

In [ ]:

!pip install chromadb


In [ ]:
import os
from typing import List, Any
import chromadb
import chromadb.config

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


class ChromaVectorStore:
    def __init__(
        self,
        persist_dir: str = "chroma_store",
        embedding_model_name: str = "all-MiniLM-L6-v2",
        chunk_size: int = 1000,
        chunk_overlap: int = 200,
    ):
        self.persist_dir = persist_dir
        os.makedirs(self.persist_dir, exist_ok=True)

        # Correctly initialize embedding model
        self.embedding_function = SentenceTransformerEmbeddings(
            model_name=embedding_model_name
        )

        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

        # Load existing DB if present
        self.vectorstore = Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding_function,
        )

        print(f"[OK] Using embedding model: {embedding_model_name}")

    def build_from_documents(self, documents: List[Any]):
        if not documents:
            raise ValueError("No documents provided")

        print(f"[INFO] Building vector store from {len(documents)} documents")

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
        )

        chunks = splitter.split_documents(documents)

        self.vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=self.embedding_function,
            persist_directory=self.persist_dir,
        )

        self.vectorstore.persist()
        print(f"[OK] Vector store saved to {self.persist_dir}")

    def query(self, query_text: str, top_k: int = 5):
        if not self.vectorstore:
            raise RuntimeError("Vector store not initialized")

        print(f"[INFO] Querying for: {query_text}")

        return self.vectorstore.similarity_search(query_text, k=top_k)


In [1]:
!pip install sentence-transformers
# Initialize vector store
chroma_store = ChromaVectorStore()

# Build only if empty
if chroma_store.vectorstore._collection.count() == 0:
    chroma_store.build_from_documents(pdf_documents)
else:
    print("[INFO] Using existing Chroma DB")

# Query
query = "Explain attention mechanism in transformer neural networks"
query_results = chroma_store.query(query, top_k=2)

# Display results
if not query_results:
    print("No relevant documents found.")
else:
    for i, doc in enumerate(query_results, 1):
        print(f"\nResult {i}:")
        print(doc.page_content)
        print("Metadata:", doc.metadata)


  Using cached sentence_transformers-5.2.0-py3-none-any.whl.metadata (16 kB)
Using cached sentence_transformers-5.2.0-py3-none-any.whl (493 kB)


NameError: name 'ChromaVectorStore' is not defined